# Molecular Dynamics

In molecular dynamics the motion of a set of atoms is determined from a model for the inter-atom interactions.
The simulations produce a set of trajectories for all the atoms in the model.
Statistical mechanics allows us to go from the atomic hypothesis to theories for macroscopic behaviour.
However, we can also study the behaviour of a system in detail by calculating and following the motion of each atom
in the system. Also, molecular dynamics allows you to pose and answer many questions about both atomic scale systems
and many-particle systems. This tool is used to understand and engineer chemical processes such as catalysis,
to model,and understand biological procesess, and to understand the interaction between drugs and living organisms.  
  
Determining the motion of a system of atoms is really a quantum mechanical problem. The motion and interaction of atoms,
including the full motion of the electrons, needs to be determined from the basic equations of quantum mechanics.
To address the behaviour of macroscopic systems, we need thousands, millions, or billions of atoms to get representative results from our calculations. Then we require a more computationally efficient approach.  
The use of the classical approximation to describe the motion of a system of atoms is, fortunately, able
to capture many of the features of a macroscopic system. In this approximation we use Newton's laws of motion
and a well-chosen description of the forces between atoms to find the motion of the atoms.
The forces are described by the potential energies of the atoms given their positions.
The potential energy functions can, in principle, be determined from quantum mechanical calculations, because the forces and potential energies depend on the states of the electrons. It is the electronics that form the bonds between atoms
and are the origin of forces between atoms.
However, in the classical approximation we parameterize: we construct simplified models for the interactions
that provide the most important features of the interactions.

# Macroscopic observables

We can use MD simulation to measure macroscopic quantities by averaging properties of the simulation system.
The ergodic hypothesis states that the time a system has one particular value of an observable $A$ is proportional
to the phase space volume where $A$ has this value. This applies to systems in equilibrium studied for
a long period of time. As a result, the time average and the ensemble average of a variable are equal.
If we average over long enough periods of time, and our microscopic interaction model is correct,
we can use the models to gain insight into the statistical properties of systems with many particles,
and in particular the fluctuations in such systems.

# 1) Molecular dynamics algorithms:
Potentials, integration, cutoff, periodic boundary conditions, initialization, efficiency improvements.
  
**Potentials**:  
The behaviour of a system will depend on the potential energy, and how it depends on the positions
of all the particles. In general we assume the potential energy $U$ depends on the positions $\boldsymbol{r}_i$
of all the particles:  
$$ U = U( \{ \boldsymbol{r}_i \} ) $$  
In general, the potential energy may depend on additional internal states of the interactions between the atoms.
For example we may know that two atoms interact through a covalent bond, and may therefore prescribe this covalent
bond as part of the potential energy. The energy would depend on both the positions of the particles,
and which particles are connected by these covalent bonds.  
We consider two types of interactions: 1) bonded interactions where the interaction types are prescribed, and
do not change throughout the simulation 2) non-bonded, open for modeling more complicated interactions, including
the formation and breaking of strong bonds.  
  
We assume the potential energy can be written as a sequence of interactions of higher and higher order,
starting from pair-wise interactions, then moving on to three-particle, four-particle interactions and so on.  
  
The parametrized models can be simple or complicated. They can include only pair interactions, three-particle
interactions, or even many-particle interactions. We will primarily use simple models, since the statistical
and thermal effects we are interested in do not depend strongly on the details of the system.
One of the simplest models for atom-atom interactions is a representation of the interactions between noble-gas
atoms, such as between two Argon atoms. For the interaction between two noble-gas atoms we have two main contributions:  
*attractive force* due to a dipole-dipole interaction. The potential for this interaction is proportional
to $(1/r)^6$, where $r$ is the distance betweent the atoms. This interaction is called the van der Waals interaction,
and we call the corresponding force the van der Waals force.  
*repulsive force*, which is a quantum mechanical effect due to the possibility of overlapping electron orbitals
as the two atoms are pushed together. We use a power law of the form $(1/r)^n$ to represent this interaction.
It is common to use $n = 12$ since $(1/r)^{12} = ((1/r)^6)^2$, which gives a good approximation for the behaviour
of Argon.  
The combined model is called the Lennard-Jones potential:  
$$ U(r) = 4\epsilon ( ( \frac{\sigma}{r} )^{12} - ( \frac{\sigma}{r} )^6 ) .$$  
Here $\epsilon$ is a characteristic energy, and $\sigma$ is a characteristic length, which are both
specific for the atoms we are modelling.  
The Lennard-Jones potential is often used not only as a model for a noble gas, but as a fundamental model
that reproduces behaviour that is representative for systems with many particles. Lennard-Jones models
are often used as base building blocks in many interatomic potentials, such as for the interaction between
water molecules and methane and many other systems where the interactions between molecules or between molecules
and atoms is simplified (coarse-grained) into a single, simple potential. Using the LJ-model you can model
$10^2$ to $10^6$ atoms on your laptop, and we can model $10^{10} - 10^{11}$ atoms on large supercomputers.
We can also model other systems, such as water, graphene or complex biological systems using this or other potentials.  
  
**Three-particle potentials**:  
In a two-particle potential the direction of bonds is not specified. The effect of covalent bonds can be
introduced by using a three-particle potential. Three-particle potentials depend on the relative positions
of three atoms, such as atom $i$ and its neighbors $j$ and $k$. For a bonded potential, we prescribe
which atoms are connected by three-particle interactions. For example we can introduce a potential term
that depends on the angle $\theta_{ijk}$ between $i$, $j$ and $k$. The corresponding term could be:  
$$ V_{ijk} = \epsilon_{ijk}(\cos{\theta_{ijk}} - \cos{\theta_{ijk,0}})^2 ,$$
where $\epsilon_{ijk}$ is the energy for this interaction and $\theta_{ijk,0}$ is the equilibrium angle
between the three atoms. This is not only dependent on the positions, but the *state* - a description
of what atoms are interacting with each other. A cut-off function can be introduced that depends on the distances,
so that the angular dependency is only included if the atoms are close together (often exponential).  
Stillinger-Weber has two- and three-particle interactions, and all the parameters depend on the type of atom.
It is a good model for silicon, Si, and can also be a good approximation for structures involving Si.
  
**Initialization**:  
An atomic (molecular) dynamics simulation starts from an initial configuration of atoms and determines the
trajectories of all the atoms. The initial condition for such a simulation consists of all the positions
$\boldsymbol{r}_i(t_0)$ and velocities $\boldsymbol{v}_i(t_0)$ at the initial time $t_0$.
In order to model a realistic system, it is important to choose the initial configuration with some care.
Since most potentials such as the Lennard-Jones potential increase very rapidly as the interatomic distance $r$
goes to zero, it is important not to place the atoms too close to each other. We therefore often place the atoms
regularly in space, on a lattice, with initial random velocities.  
We generate a lattice by first constructing a *unit cell*, and then copying this unit cell to each position
of a lattice to form a regular pattern of unit cells (the unit cell may contain more than one atom).  
For a cubic unit cell of length $b$ with only one atom in each unit cell, we can place the atoms
at $(0,0,0)$ in the unit cell and generate a cubic lattice with distances $b$ between the atoms
by using this cubic unit cell to build a lattice. Such a lattice is called a *simple cubic lattice*.  
For a Lennard-Jones system we know (from other theoretical, numerical and experimental studies) that
the equilibrium crystal structure is not a simple cubic lattice, but a face centered cubic lattice.
This is a cubic lattice, with additional atoms added on the center of each of the faces of the cubes.
We use this as our basis and select a lattice size $b$ so that we get a given density of atoms.
The whole system consists of $L \times L \times L$ cells, each of size $b \times b \times b$
and with 4 atoms in each cell.  
  
**Maxwell-Boltzmann**:  
Probability distribution first defined and used for describing particle speeds in idealized gases
where the particles move freely inside a stationary container without interacting with one another, 
except for very brief collisions in which they exchange energy and momentum with each other or with their
thermal environment. System assumed to have reached thermal equilibrium. Applies to the classical ideal gas.
In real gases there are various effects that make their speed distribution different.
  
**Boundary conditions**:  
Macroscopically large systems contain on the order of $10^{23}$ atoms, which is simply beyond practical computational capabilities. Is it possible to gain insights into how very large, even infinite, systems behave?
One of the problems with a small system is the external boundaries. We can "fool" the system into believing
it is infinite by applying what we call periodic boundary conditions (PBC). If the atoms on the left
of the system do not see emptiness on their left, but instead see the right hand side of the system,
as if the system is wrapped around a cylinder, the system will look like it is infinite.
The convention of PBC is usually applied in all simulations in order to avoid dealing with boundary conditions.
However, PBC can also introduce correlational artifacts that do not respect the translational invariance
of the system. The strain field arising from any inhomogeneity will be artificially truncated and modified
by the periodic boundary, and the sound or shock waves and phonons in the system are limited by the box size.
Net electrostatic charge must be zero to avoid summing to an infinite charge. Minimum-image convention
also generally requires that a spherical cutoff radius for nonbounded forces
be at most half the length of one side of a cubic box.
In a box that is too small, a macromolecule may interact with its own image in a neighboring box, like
a molecule's head interacting with its own tail, and produces highly unphysical results.  
  
**Equations of motion**:  
For molecular dynamics simulations we usually use an algorithm called the Velocity-Verlet, which is approximately
like the forward-Euler method, but very well suited for conservative forces. The velocity is calculated
at both time $t$ and at intermediate times $t + \Delta t/2$, where $\Delta t$ is the time-step,
whereas the forces are only calculated at full time-steps $t, t + \Delta t, t + 2\Delta t$, etc.
The most time-consuming part of the calculation is the calculation of the forces.
We therefore want to limit the number of times we calculate the forces, and still have as high precision
as possible in our calculations. Velocity-Verlet is a good trade-off between precision in the integration algorithm
and the number of times forces are calculated. Positions $\boldsymbol{r}_i(t)$ and velocities $\boldsymbol{v}_i(t)$
of all the atoms $i = 1,...,N$ are propagated forward in time.
This method has very good properties when it comes to energy conservation, and it does preserve momentum perfectly.  
  
**Non-dimensional equations of motion**:  
All the quantities in a molecular dynamics simulation are very small. Usually we introduce measurement units that
are adapted to the task. For LJ we use the intrinsic length and energy scales of the model as the basic units.
Measure length in $\sigma$ and energies in $\epsilon$. A vector $\boldsymbol{r}_i '$ in the simulation
is related to the real-world length as:  
$$ \boldsymbol{r}_i ' = \boldsymbol{r}_i / \sigma .$$
Can also introduce a LJ time $\tau = \sigma \sqrt{m / \epsilon}$, where $m$ is the mass of the atoms
and the LJ temperature $T = \epsilon / k_B$.  
  
**Constraints/cutoff/efficiency**:  
The most CPU-intensive task is the evaluation of the potential as a function of the particles
internal coordinates. The most expensive one is the non-bonded or non-covalent part. If all pair-wise
interactions are accounted for they scale by $\mathcal{O}(n^2)$.
Particle mesh ewald summation scales as $\mathcal{O}(n \log{n})$ (summation on a discrete mesh in Fourier space
with truncation). Particle-Particle-Particle-Mesh (P3M), where you use a direct sum for close particles, or
good spherical cutoff methods can take this down to $\mathcal{O}(n)$.  
The time-step must be small enough to avoid discretization errors, order of femtoseconds $10^{-15}$s.
Cutoff can be computed efficiently using neighbor lists: build lists every few timesteps and in other timesteps
scan larger list for neighbors within cutoff radius. Link-cells (bins) create a grid domain with bins of
size $R_{force}$, and at each step searches a number of bins for neighbors. LAMMPS uses link-cells to build Verlet
list, and use the Verlet list on non-build timesteps.
For parallelization, physical domain divided into 3d boxes, one (or more) per processor. Each processor
computes forces on atoms in its box using ghost info from nearby processors. Atoms carry along molecular topology
as they migrate to new processors. Scales sub-linear.

# 2) Molecular dynamics in the micro-canonical ensemble:
Initialization and initialization effects. Temperature measurements and fluctuations. Use of thermostats for
initialization.  
  
**Initial conditions**:  
The equilibrium state of the system will not depend on the initial conditions unless there are local minima
in the Gibbs Free Energy (metastable states).  
For the simulation of fluids, we generally begin the system in a perfect simple cubic structure or
face-centered-cubic (FCC) lattice. The lattice constants are such that the atoms are equally distributed
throughout the entire simulation volume. As long as the temperature of the simulation is above the melting temperature
of the material, the material will gradually "melt" out of this initial configuration. This is simple and better
than random, since there is no chance of overlap between atoms. Overlap can give rise to highly repulsive forces
which cause the simulation to "blow-up" in the first few timesteps.  
For velocities we enforce that translational momentum must be conserved and the kinetic energy must be related
to the thermodynamic equipartition theorem $\langle E \rangle = (3/2) N k_b T$.
The equipartition theorem makes accurate predictions in certain conditions, but is inaccurate
when quantum effects are significant, such as at low temperatures (ex. heat capacity of a solid decreases
at low temperatures, rather than remaining constant as in equipartition).
Optionally, the velocity distribution should follow the Maxwell-Boltzmann distribution. As the system equilibriates
this should be automatically fulfilled, and satisfying it initially might speed up the process of equilibration.
Important to choose a configuration that can relax quickly to the structure and velocity distribution
appropriate.

**Temperature**:  
Temperature is a state variable that specifies the thermodynamic state of the system. It is related
to the microscopic description of molecular simulations through the kinetic energy.
Temperature is a thermodynamic quantity, which is only meaningful at equilibrium.
Temperature is usually measured using the equipartition theorem, $T = (2/3) \frac{\langle E \rangle}{N k_b}$.
This assumes equilibrium between the translational and potential degrees of freedom. The measurements of the kinetic
energy should only be taken after the system is equilibriated, although plotting the temperature
as a function of time can be a method of finding the equilibration time of the system.
If our initial configuration has a higher potential energy than the equilibrium state, then as we equilibrate the
potential energy drops, and due to conservation of energy kinetic energy (and consequently the temperature)
increases. The opposite is also true. If we would like to simulate a particular temperature, we would need
to scale the velocities so that the temperature is constant. Scaling changes energy, but this can be done
during the equilibration steps. Temperature will fluctuate after equilibration, since energy is conserved,
but it will fluctuate around the temperature we scaled to. Note that the error in the measurement
of extensive quantities scales as $1/\sqrt{n}$.
  
**Thermostats**:  
If we wish to study systems in contact with a heat bath (a common situation in physics) we need
to modify the velocity Verlet algorithm to sample from the NVT ensemble (where the temperature is fixed)
instead of the NVE ensemble (where the energy is fixed). Microcanonical NVE $\rightarrow$ canonical NVT.  
Requirements for a good thermostat are:  
1) Keeping the system temperature around the heat bath temperature  
2) Sampling the phase space corresponding to the canonical ensemble  
3) Tunability  
4) Preservation of dynamics  
  
*Nose-Hoover*: This thermostat introduces a fictious dynamical variable, whose physical meaning is that of a
friction $\xi$, which slows down or accelerates particles until the temperature is equal to the desired value.
Temperature is therefore not fixed, but tends to the target value. The equations of motion can be implemented
by a small modification to the velocity Verlet algorithm. This is the default in LAMMPS.
Another option is to use a stochastic thermostat. This thermostat again requires the introduction of dissipative
forces, through friction, which physically comes from fluctuating forces on a moving particle due to
the chaotic motion of solvent molecules. We therefore also need to include a stochastic term to account for such
fluctuating forces. The equations with a stochastic thermostat are known as Brownian dynamic equations.  
  
*Andersen*: The Anderson thermostat couples the system to a heat bath via stochastic forces that modify the kinetic energy of the atoms or molecules. The time between collisions, or the number of collisions in some (short)
time interval is decided randomly, with a Poisson distribution $P(t) = \nu e^{-\nu t}$,
where $\nu$ is the stochastic collision frequency. Between collisions the the system evolves at constant energy.
Upon a collision event the new momentum of the lucky atom is chosen at random from a Boltzmann distribution
at temperature $T$. In principle $\nu$ can adopt any value, but there exists an optimal choice. This thermostat
should only be used for time-independent properties, and dynamic properties such as diffusion should not be
calculated while the system is thermostated using the Andersen algorithm.  
  
*Berendsen*: The Berendsen thermostat is an algorithm to re-scale the velocities of particles in molecular dynamics
to control the simulation temperature. The thermostat uses a weak coupling to an external heat bath of temperature
$T_0$. This results in the modified equation of motion:  
$$ m_i \frac{d\boldsymbol{v}_i}{dt} = \boldsymbol{F}_i + m_i \gamma (\frac{T_0}{T} - 1) \boldsymbol{v}_i .$$
This represents a proportional scaling of the velocities per time step from 
$\boldsymbol{v}$ to $\lambda \boldsymbol{v}$, where  
$$ \lambda = [ 1 + \frac{\Delta t}{\tau_T}(\frac{T_0}{T} - 1)]^{1/2} ,$$
where $\tau_T$ is a time constant associated with the coupling.  
The thermostat suppresses fluctuations of the kinetic energy of the system, and therefore cannot produce
trajectories consistent with the canonical ensemble. For large systems the approximation yields roughly correct results
for most calculated properties. This scheme is widely used due to the efficiency. Many systems are equilibriated
using Berendsen, while properties are calculated using Nose-Hoover.

# 3) Measurements of macroscopic quantities:
Temperature and pressure. Measurements in LAMMPS. Challenges? Uses? Examples from own simulation.  
  
*Temperature*: The *instanteneous* temperature is calculated using the equipartition theorem, relating
it to the kinetic energy degrees of freedom. The number of atoms is assumed to be constant for the duration.
Temperature must be measured in equilibrium, and can be averaged over to obtain the average temperature.
We require that there be a large enough number of atoms. The equipartition theorem is inaccurate when
quantum effects are significant, such as at low temperatures. When the thermal energy $k_B T$ is smaller
than the quantum energy spacing in a particular degree of freedom, the average energy and heat capacity
of this degree of freedom is much smaller than this spacing. Such a degree of freedom is said to be
"frozen out". Do not know how this affects MD.  
  
*Pressure*: The pressure is computed using the virial theorem:  
$$ PV = N k_B T + \sum F_{ij} \cdot r_{ij} .$$
The second term is the virial, equal to $-dU/dV$, computed for all pairwise, as well as 2-body, 3-body, 4-body,
manybody and long-range interactions. When PBC are used, $N$ necessarily includes periodic image (ghost) atoms
outside the central box, and the position and force vectors of ghost atoms are thus included in the summation.
Fixes that impose constraints also contribute to the virial term. Pressure is a macroscopic quantity, and can only be measured as a time average.  
  
Molecular dynamics is used extensively in modern computational protein-folding, and other simulations
of biological processes. It has also been used to investigate the effect of surface charges on disjoining
pressure of thin water films on metal surfaces.

# 4) Measuring the diffusion constant:  
Limitations, challenges and results. Comparison with theoretical estimates for diffusive behaviour.
Difference between potential models.  
  
The self-diffusivity describes the random motion of a molecule in the absence of any gradients that would cause
a mass flux. Self-diffusion coefficients can be obtained from molecule positions or velocities. Obtaining it from the velocities involve integrating the velocity auto-correlation function, an example from what is called the Green-Kubo relations. The long-time tail to this integral can cause numerical problems. Einstein related the SD coefficient to the
mean square displacement of a particle as a function of observation time. The mean square displacement is proportional
to the observation time in the limit the observation time goes to infinity:  
$$ D = \frac{1}{2d} \lim_{t \rightarrow \infty} \frac{\langle [r(t_0 + t) - r(t_0)]^2 \rangle}{t} ,$$
where $D$ is the SD coefficient and $d$ is the dimensionality of the system. The numerator is the MSD, the angled brackets indicate an ensemble average has been taken. The ensemble average is an average over all molecules in the simulation and all origins. By origins we mean that any timestep can be considered the time zero in the equation,
since the equation is only looking at observation times. There is no general rule for how long a simulation has to be run before we reach the long time asymptotical behaviour. Positions are stored at every timestep for every molecule.
The sample average is computed over both molecules and "time origins". Any point during the simulation can be considered a time origin, however, we might like all times to be represented equally in terms of the number of data points used
to generate the MSD versus the time plot (e.g. first $50 \%$).
Once we have the MSD we can simply perform linear regression. Since there is some unknown short time behaviour,
the linear portion should be shifted so that the intercept is non-zero.
In an isotropic medium one can average over x, y, z to obtain the average self-diffusion constant.
Plotting the log-log behaviour, one can find three regions of behaviour.  
1) Free motion. Very short observation time before any collisions have occured.  
2) Intermediate, MSD proportional to observation time raised to some power between 1 and 2  
3) Long time behaviour  

# 5) Interaction models:  
Introduce the three potential models Lennard-Jones, Stillinger-Weber and SPC/E. Discuss similarities and
differences in your results, with particular emphasis on initialization, efficiency, radial distribution $g(r)$
and diffusion $D(T)$. Provide examples from own simulations.  
  
**Stillinger-Weber**:  
Silicon forms 4-coordinated tetrahedral bonded structures. The object of the three-body component of the potential
is to enforce the tetrahedral bond angle ($109.47 \,^{\circ}$) among triplets of bonded atoms.
The total potential is expressed as two sums, one for unique pair interactions and another for unique triplet
interactions. The two-body part is very much like a LJ potential, only with different exponents and a smooth cutoff,
as the interatom separation distance approaches some cutoff $a$ exponentially.
The three-body part models the angles, and is the sum of functions of each of the three angles of a triplet.
The system melts without including the three-body part, which demonstrates that one must have three-body forces to
stabilize a diamond-cubic lattice.  
  
**Water models**:  
The behaviour of water is both important and complex. Important because of its role in biological and geological
processes and chemical processing. Complex because of the many non-trivial details in the behaviour of water:
the equations of state, the structure of water and ice, the effect of weak hydrogen bonding, critical points and phase
transitions and heat capacity. Many models have been developed with focus on different physical effects with a large variation in complexity. Simple models capture some effects and allow large simulations, whereas more complex models capture many effects, but are limited to small systems.  
The models typically include both electrostatic effects from point charges and Lennard-Jones effects.
The Lennard-Jones effects ensure that there are repulsive interactions at short distances so that the structures do not collapse under electrostatic interactions. The electrostatic interactions introduce a component that leads to alignment
and directional interactions. Typically the models are developed to ensure a good fit to one or a few physical structures or parameters, such as the radial distribution function or critical parameters. Good fit for the radial distribution when compared with experiment is not a sensitive test, but mainly ensure tetrogonal structure.  
  
**SPC and SPC/E**:  
The simplest water models are rigid models, where the relative positions of the atoms in the water molecule are constant.
The interactions between molecules are in the form Lennard-Jones and electrostatic forces:  
$$ U = \sum \frac{k_c q_i q_j}{r_{ij}} + 4\epsilon ((\frac{\sigma}{r_{OO,ij}})^{12} - (\frac{\sigma}{r_{OO,ij}})^6) ,$$
where the $r_{ij}$ distances are between point charges in the two molecules and $r_{OO,ij}$ are between the oxygen atoms
in the two molecules. Many models use the actual angle between the hydrogen atoms, but SPC uses a slightly higher angle.
The Single Point Charge (SPC) models in their simplest form include LJ interactions and the effect of single point charges placed at the positions of the oxygen and hydrogen atoms. The SPC/E model is a modified version that also
includes an additional average polarization correction in the energy. TIP3P is also a 3-site model, which is slightly modified in the CHARMM force field. Flexible SPC explicitly includes the behaviour of the O-H and H-O-H bonds and the bonds are modeled. The O-H stretching can be made anharmonic, which provides a better description of the dynamical behaviour of the system (considered one of the most accurate without polarization effects).  
  
In LAMMPS we use SPC/E and the water molecules move as a rigid body. O-H bonds and H-O-H angles are held constant.